# First Machine Learning Model

## Selecting Data for Modeling

Your dataset had too many variables to wrap your head around, or even to print out nicely. How can you pare down this overwhelming amount of data to something you can understand?

We'll start by picking a few variables using our intuition. Later courses will show you statistical techniques to automatically prioritize variables.

To choose variables/columns, we'll need to see a list of all columns in the dataset. That is done with the **columns* property of the DataFrame (the bottom line of code below).

In [1]:
import pandas as pd

melb_file_path = '../input/melbourne-housing/melb_data.csv'
melb_data = pd.read_csv(melb_file_path) 
melb_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

The Melbourne data has some missing values, in this case some houses for which some variables weren't recorded.

We'll learn to handle missing values in a later tutorial and your Iowa data doesn't have missing values in the columns you use. So we will take the simplest option for now, and drop houses from our data. 

Don't worry about this much for now, though the code is:

In [4]:
# dropna drops missing values (think of na as "not available")
melb_data.dropna(axis=0, inplace=True)
melb_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0
6,Abbotsford,124 Yarra St,3,h,1876000.0,S,Nelson,7/05/2016,2.5,3067.0,...,2.0,0.0,245.0,210.0,1910.0,Yarra,-37.8024,144.9993,Northern Metropolitan,4019.0
7,Abbotsford,98 Charles St,2,h,1636000.0,S,Nelson,8/10/2016,2.5,3067.0,...,1.0,2.0,256.0,107.0,1890.0,Yarra,-37.8060,144.9954,Northern Metropolitan,4019.0


There are many ways to select a subset of your data. The Pandas Micro-Course covers these in more depth, but we will focus on two approaches for now.
1. Dot notation, which we use to select the "prediction target"
2. Selecting with a column list, which we use to select the "features"

### Selecting The Prediction Target

You can pull out a variable with dot-notation. This single column is stored in a **Series**, which is broadly like a DataFrame with only a single column of data.

We'll use the dot notation to select the column we want to predict, which is called the **prediction target**.
By convention, the prediction target is called **y**. So the code we need to save the house prices in the Melbourne data is:

In [6]:
y = melb_data.Price
y.describe()

count    6.196000e+03
mean     1.068828e+06
std      6.751564e+05
min      1.310000e+05
25%      6.200000e+05
50%      8.800000e+05
75%      1.325000e+06
max      9.000000e+06
Name: Price, dtype: float64

### Choosing "Features"

The columns that are inputted into our model (and later used to make predictions) are called **features**. In our case, those would be the columns used to determine the home price. Sometimes, you will use all columns except the target as features. Other times you'll be better off with fewer features.

For now, we'll build a model with only a few features. Later on you'll see how to iterate and compare models built with different features.

We select multiple features by providing a list of column names inside brackets. Each item in that list should be a string (with quotes).

Again by convention, the **features** data is called X. Here is an example:

In [12]:
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
X = melb_data[melbourne_features]

Let's quickly review the data we'll be using to predict house prices using the describe method and the head method, which shows the top few rows.

In [13]:
X.describe()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,-37.807904,144.990201
std,0.971079,0.711362,897.449881,0.075850,0.099165
min,1.000000,1.000000,0.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,-37.457090,145.526350


In [15]:
X.head(3)

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
1,2,1.0,156.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,144.9944
4,4,1.0,120.0,-37.8072,144.9941


Visually checking your data with these commands is an important part of a data scientist's job. In this first look you'll frequently find surprises in the dataset that deserve further inspection.

## Building Your Model

You will use the **scikit-learn** library to create your models. When coding, this library is written as **sklearn**, as you will see in the sample code. 

**Scikit-learn** is the most popular library for modeling the types of data typically stored in DataFrames.

The steps to building and using a model are:
- **Define**: What type of model will it be? A decision tree? Some other type of model? Some other parameters of the model type are specified too.
- **Fit**: Capture patterns from provided data. This is the heart of modeling.
- **Predict**: Just what it sounds like
- **Evaluate**: Determine how accurate the model's predictions are.

Here is an example of defining a **Decision Tree Regressor** model with **scikit-learn** and fitting it with the features and target variables.

In [16]:
from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
melb_model = DecisionTreeRegressor(random_state=17)

# Fit model
melb_model.fit(X, y)

DecisionTreeRegressor(random_state=17)

Many machine learning models allow some randomness in model training. Specifying a number for **random_state** ensures you get the same results in each run. This is considered a good practice. You may use any number, and model quality won't depend meaningfully on exactly what value you choose.

We now have a fitted model that we can use to make predictions. In practice, you'll want to make predictions for new houses coming on the market rather than the houses we already have prices for.

But just for illustration purposes, here we'll make predictions on the first few rows of the training data to see how the predict function works.

In [33]:
df = pd.DataFrame(X)
df['y'] = y
dfs = df.sample(n=5)
yp = dfs.y
Xp = dfs.drop('y', axis=1)

print("Making predictions for the following 5 houses:")
print(Xp)

print("The predictions are")
print(melb_model.predict(Xp))

print("The real prices are:")
print(list(yp))

Making predictions for the following 5 houses:
       Rooms  Bathroom  Landsize  Lattitude  Longtitude
12183      2       1.0     140.0  -37.68199   145.01744
10063      4       2.0     883.0  -37.76255   145.16233
10581      2       1.0       1.0  -37.93320   145.03794
1342       3       1.0     483.0  -37.76020   144.95430
3172       4       2.0     218.0  -37.81420   145.02520
The predictions are
[ 407000. 1450000.  380000. 1250000. 1936000.]
The real prices are:
[407000.0, 1450000.0, 380000.0, 1250000.0, 1936000.0]
